In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import stan
import arviz as az
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Data Preparation
omega = pd.read_csv('../EpiOS/omega.csv')['omega'].values
theta = pd.read_csv('../EpiOS/y_interpolated.csv')['y_interpolated'].values


In [3]:
# This correspond to 3 and problem 1
poisson_model = """
functions {
    real effective_no_infectives (
        int N, int S, int t, array [] real aI, array [] real aOmega) {
            real mean;
            if(t > S) {
                mean = (
                    dot_product(aI[(t-S):(t-1)], aOmega));
            }
            else {
                mean = (
                    dot_product(aI[:(t-1)], aOmega[(S-t+2):]));
            }
            return mean;
    }
}
data {
    int N; // number of days
    int S; // length omega
    array [N] real Theta; // local prevalence for N days
    array [S] real revOmega; // reversed PCR positive probability
}
parameters {
    array [N] real<lower=0> I; // local incidence for N days
    real<lower=0> sigma; // variance of the log-normal distribution
}
model {
    for(t in 1:N) {
        Theta[t] ~ normal (
            effective_no_infectives(
                N, S, t, I, revOmega), sigma); // likelihood
    }
    sigma ~ normal(0, 1); // prior of sigma
    for(t in 1:N) {
        I[t] ~ normal(5000, 1700); // prior of I
    }
}
"""
poisson_data = {
    'N': len(theta),
    'S': len(omega),
    'Theta': theta, #  change to frequency
    'revOmega': omega[::-1]}

In [4]:
# Correspond to 2
posterior = stan.build(poisson_model, data=poisson_data)
fit = posterior.sample(num_chains=3, num_samples=1000)

# Assuming time_sample correctly indexes the days for 'I' and 'Theta'
samples = az.from_pystan(
    posterior=fit,
    observed_data='Theta',  # Use 'Theta' as the observed data
    coords={'observation': list(range(len(theta)))},  # 'covariate' removed
    dims={'Theta': ['observation'], 'I': ['observation']}  # Adjusted dims
)
df = az.summary(samples)
df

Building...

Error handling request
Traceback (most recent call last):
  File "/Users/yunliqi/mambaforge/envs/test/lib/python3.11/site-packages/aiohttp/web_protocol.py", line 452, in _handle_request
    resp = await request_handler(request)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yunliqi/mambaforge/envs/test/lib/python3.11/site-packages/aiohttp/web_app.py", line 543, in _handle
    resp = await handler(request)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yunliqi/mambaforge/envs/test/lib/python3.11/site-packages/httpstan/views.py", line 103, in handle_create_model
    _, stanc_warnings = httpstan.compile.compile(program_code, stan_model_name)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yunliqi/mambaforge/envs/test/lib/python3.11/site-packages/httpstan/compile.py", line 37, in compile
    completed_process = subprocess.run(run_args, capture_output=True, timeout=1)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

ValueError: TAPE_ERROR: The JSON document has an improper structure: missing or superfluous commas, braces, missing keys, etc.